In [1]:
%load_ext autoreload
%autoreload 2

Load dataset

In [2]:
import torch
from lerobot.common.datasets.lerobot_dataset import LeRobotDataset

repo_id = "lerobot/aloha_sim_transfer_cube_human_image"

# Load a subset of episodes
# Add more action sequence to the dataset
delta_timestamps = {
    # loads 4 action vectors
    "action": [t / 50 for t in range(4)]
}
dataset = LeRobotDataset(
    repo_id, episodes=[0, 1, 2, 3], delta_timestamps=delta_timestamps
)

print(f"Selected episodes: {dataset.episodes}")
print(f"Number of frames selected: {dataset.num_frames}")
camera_key = dataset.meta.camera_keys[0]
print(camera_key)


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Selected episodes: [0, 1, 2, 3]
Number of frames selected: 1600
observation.images.top


In [3]:
from pprint import pprint
# pprint(dataset.features)
print(dataset.fps)
print(f"{dataset[0]['observation.state'].shape=}")
print(f"{dataset[0]['action'].shape=}")
print(f"{dataset[0]['observation.images.top'].shape=}")

50
dataset[0]['observation.state'].shape=torch.Size([14])
dataset[0]['action'].shape=torch.Size([4, 14])
dataset[0]['observation.images.top'].shape=torch.Size([3, 480, 640])


In [4]:

dataloader = torch.utils.data.DataLoader(
    dataset,
    num_workers=4,
    batch_size=1,
    shuffle=False,
    pin_memory=True,
)

for batch in dataloader:
    print(f"{batch['observation.images.top'].shape=}")  # (1, 4, c, h, w)
    print(f"{batch['observation.state'].shape=}")  # (1, 5, c)
    print(f"{batch['action'].shape=}")  # (1, 64, c)
    print(f"{batch.keys()=}")
    break

batch['observation.images.top'].shape=torch.Size([1, 3, 480, 640])
batch['observation.state'].shape=torch.Size([1, 14])
batch['action'].shape=torch.Size([1, 4, 14])
batch.keys()=dict_keys(['observation.images.top', 'observation.state', 'action', 'episode_index', 'frame_index', 'timestamp', 'next.done', 'index', 'task_index', 'action_is_pad'])


In [5]:
from models import CVAE
from torch.optim import Adam

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = CVAE().to(device)
optimizer = Adam(model.parameters(), lr=1e-4)

/home/sledge/miniconda3/envs/deep-learning/lib/python3.10/site-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
/home/sledge/miniconda3/envs/deep-learning/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/sledge/miniconda3/envs/deep-learning/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
# Run training loop.
from tqdm import tqdm


def train(model: CVAE, optimizer, train_loader, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        for batch in (pbar := tqdm(train_loader, desc=f"Epoch {epoch}/{num_epochs}")):
            batch = {k: v.to(device) for k, v in batch.items()}
            predicted_actions, mu_phi, logvar = model(
                batch['observation.state'],
                batch['action'].squeeze(),
                batch['observation.images.top']
            )
            optimizer.zero_grad()
            loss = model.loss_fn(
                predicted_actions, batch['action'].squeeze(), mu_phi, logvar)
            loss.backward()
            optimizer.step()

            pbar.set_postfix({'loss': loss.item()})

In [7]:
train(model, optimizer, dataloader, num_epochs=10)

Epoch 0/10:   0%|          | 0/1600 [00:00<?, ?it/s]/home/sledge/Vision-Projects/Action-Chunking-Transformer/models.py:174: UserWarning: Using a target size (torch.Size([1, 4, 14])) that is different to the input size (torch.Size([4, 14])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  reconstruction = F.mse_loss(predict, target, reduction='mean')
Epoch 9/10: 100%|██████████| 1600/1600 [01:18<00:00, 20.31it/s, loss=0.00361]


In [11]:
import torch
# torch.save(model.state_dict(), 'checkpoints/cvae.pth')
model = CVAE()
model.load_state_dict(torch.load('checkpoints/cvae.pth'))
model = model.to(device)
model.eval()

/home/sledge/miniconda3/envs/deep-learning/lib/python3.10/site-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
/home/sledge/miniconda3/envs/deep-learning/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/sledge/miniconda3/envs/deep-learning/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/

CVAE(
  (encoder): ActionEncoder(
    (transformer_encoder): TransformerEncoder(
      (layers): ModuleList(
        (0-1): 2 x TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
          )
          (linear1): Linear(in_features=128, out_features=2048, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=2048, out_features=128, bias=True)
          (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (joints_mlp): Linear(in_features=14, out_features=128, bias=True)
    (actions_mlp): Linear(in_features=14, out_features=128, bias=True)
  )
  (mu_phi): Linear(in_features=128, out_features=128, bias=True)
  (lo

Inference using FIFO queue